## Using `SequenceConstraint` to place restrictions on `Components` within the same `ComponentDefinition`

`SequenceConstraints` are used to describe the relative position and orientation between two components within the same `ComponentDefinition`. 

A minimally useful version of this object requires the existence of:

- a parent ComponentDefinition
- Two child Components

### Four Restriction Types:
1. `SBOL_RESTRICTION_PRECEDES`: `Component_A` precedes `Component_B` 
2. `SBOL_RESTRICTION_SAME_ORIENTATION_AS`: The two `Components` have the same orrientation as each other
3. `SBOL_RESTRICTION_OPPOSITE_ORIENTATION_AS`: The two `Components` have the opposite orrientation as each other
4. `SBOL_RESTRICTION_DIFFERENT_FROM`: The two `Components` do not refer to the same `ComponentDefinition`

For more information on the `SequenceConstraint` class and its properties, check out page 36 of the [SBOL 2.3.0 specifications](https://sbolstandard.org/docs/SBOL2.3.0.pdf).

We will demonstrate `SBOL_RESTRICTION_PRECEDES` using the [Part:BBa_K174004](https://parts.igem.org/Part:BBa_K174004), a pspac promoter, designed by the The Newcastle 2009 iGEM team. The genetic device has two parts, a pspac promoter and a Lac operator, with the promoter directly preceding the operator. We can use `SBOL_RESTRICTION_PRECEDES` to model this relationship.

In [2]:
import sbol2
import sbol2.sequenceconstraint

# Create a new SBOL document
doc = sbol2.Document()

# Set the homespace (namespace) for the document
sbol2.setHomespace('https://github.com/SynBioDex/SBOL-Notebooks')


# Create a ComponentDefinition for the LacI operator
lacI_operator_comp_def = sbol2.ComponentDefinition('LacI_operator', sbol2.BIOPAX_DNA)
lacI_operator_comp_def.description = 'LacI binding site'
# The role "SO:0000057" is for an operator, as defined in the Sequence Ontology
lacI_operator_comp_def.roles = ["http://identifiers.org/so/SO:0000057"]
doc.addComponentDefinition(lacI_operator_comp_def)

# Create a ComponentDefinition for the pspac promoter
pspac_promotor_comp_def = sbol2.ComponentDefinition('pspac', sbol2.BIOPAX_DNA)
pspac_promotor_comp_def.roles = [sbol2.SO_PROMOTER]
doc.addComponentDefinition(pspac_promotor_comp_def)

# Create a ComponentDefinition for the genetic part
genetic_part_comp_def = sbol2.ComponentDefinition('BBa_K174004', sbol2.BIOPAX_DNA)
genetic_part_comp_def.description = 'pspac core promoter region'
genetic_part_comp_def.roles = [sbol2.SO_PROMOTER]
doc.addComponentDefinition(genetic_part_comp_def)

# Create a Component instance for the pspac promoter and add it to the genetic part
pspac_promoter_comp = sbol2.Component('pspac')
pspac_promoter_comp.definition = pspac_promotor_comp_def.identity
genetic_part_comp_def.components.add(pspac_promoter_comp)

# Create a Component instance for the LacI operator and add it to the genetic part
LacI_operator_comp = sbol2.Component('LacI_operator')
LacI_operator_comp.definition = lacI_operator_comp_def.identity
genetic_part_comp_def.components.add(LacI_operator_comp)

# Create a SequenceConstraint specifying that the pspac promoter precedes the LacI operator
constraint = genetic_part_comp_def.sequenceConstraints.create('Constraint1')
constraint.subject = pspac_promoter_comp.identity  # Set the subject to the identity of pspac promoter
constraint.object = LacI_operator_comp.identity    # Set the object to the identity of LacI operator
constraint.restriction = sbol2.SBOL_RESTRICTION_PRECEDES  # Use the SBOL_RESTRICTION_PRECEDES restriction


As you can see, creating a `SequenceConstraint` is very easy, as all that is needed are two `Components` within the same parent `ComponentDefinition`. For our next step, we will show off some of the functionality of `SequenceConstraint`. One useful aspect of `SequenceConstraint` is that if we have specified the `Sequences` of the individual Components we can calculate the `Sequence` of the entire Genetic part.

In [3]:
# Create the pspac sequence
pspac_sequence = sbol2.Sequence('pspac_sequence')
pspac_sequence.elements = 'ttgttgactttatctacaaggtgtggcataatgtgtgg'
pspac_sequence.encoding = sbol2.SBOL_ENCODING_IUPAC
pspac_promotor_comp_def.sequence = pspac_sequence

# Create the LacI sequence
LacI_sequence = sbol2.Sequence('LacI_sequence')
LacI_sequence.elements = 'attgtgagcgctcacaatt'
LacI_sequence.encoding = sbol2.SBOL_ENCODING_IUPAC
lacI_operator_comp_def.sequence = LacI_sequence

In [5]:
print(genetic_part_comp_def.compile)



# Validate the document to ensure compliance with SBOL standards
doc.validate()

# Save the document to an SBOL file
doc.write('sequence_constraint_example.xml')

<bound method ComponentDefinition.compile of <sbol2.componentdefinition.ComponentDefinition object at 0x0000023F6B6D3DF0>>


'Valid.'